# Language Translation with Machine Learning

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing

import os
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re

import seaborn as sns
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
# from keras.layers import Input, LSTM, Embedding, Dense
# from keras.models import Model
import torch

In [2]:
import os

os.environ['CUDA_LAUNCH_BLOCKING']="1"
os.environ['TORCH_USE_CUDA_DSA'] = "1"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
lines=pd.read_csv("Hindi_English_Truncated_Corpus.csv",encoding='utf-8')
print(lines.head())

      source                                   english_sentence  \
0        ted  politicians do not have permission to do what ...   
1        ted         I'd like to tell you about one such child,   
2  indic2012  This percentage is even greater than the perce...   
3        ted  what we really mean is that they're bad at not...   
4  indic2012  .The ending portion of these Vedas is called U...   

                                      hindi_sentence  
0  राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह कर...  
1  मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहू...  
2   यह प्रतिशत भारत में हिन्दुओं प्रतिशत से अधिक है।  
3     हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते  
4        इन्हीं वेदों का अंतिम भाग उपनिषद कहलाता है।  


In [4]:
lines=lines[lines['source']=='ted']
lines=lines[~pd.isnull(lines['english_sentence'])]
lines.drop_duplicates(inplace=True)
# Let us pick any 25000 rows from the dataset
lines=lines.sample(n=25000,random_state=42)
lines.shape

(25000, 3)

In [5]:
lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.lower())
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: x.lower())

In [6]:
lines['english_sentence']=lines['english_sentence'].apply(lambda x: re.sub("'", '', x))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: re.sub("'", '', x))

In [7]:
exclude = set(string.punctuation) # Set of all special characters
# Remove all the special characters
lines['english_sentence']=lines['english_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [8]:
remove_digits = str.maketrans('', '', digits)
lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.translate(remove_digits))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: x.translate(remove_digits))

lines['hindi_sentence'] = lines['hindi_sentence'].apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))

# Remove extra spaces
lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.strip())
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: x.strip())
lines['english_sentence']=lines['english_sentence'].apply(lambda x: re.sub(" +", " ", x))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: re.sub(" +", " ", x))

lines['hindi_sentence'] = lines['hindi_sentence'].apply(lambda x : 'START_ '+ x + ' _END')

In [9]:
### Get English and Hindi Vocabulary
all_eng_words=set()
for eng in lines['english_sentence']:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

all_hindi_words=set()
for hin in lines['hindi_sentence']:
    for word in hin.split():
        if word not in all_hindi_words:
            all_hindi_words.add(word)
lines['length_eng_sentence']=lines['english_sentence'].apply(lambda x:len(x.split(" ")))
lines['length_hin_sentence']=lines['hindi_sentence'].apply(lambda x:len(x.split(" ")))

In [10]:
lines=lines[lines['length_eng_sentence']<=20]
lines=lines[lines['length_hin_sentence']<=20]
max_length_src=max(lines['length_hin_sentence'])
max_length_tar=max(lines['length_eng_sentence'])

input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_hindi_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_hindi_words)
num_encoder_tokens, num_decoder_tokens

num_decoder_tokens += 1 #for zero padding
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())
lines = shuffle(lines)

## Training Model to Translate English to Hindi

In [11]:
X, y = lines['english_sentence'], lines['hindi_sentence']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=42)

X_train.to_pickle('X_train.pkl')
X_test.to_pickle('X_test.pkl')

In [19]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)
          
latent_dim=20
# encoder_inputs = Input(shape=(None,))
# encoder_inputs = torch.tensor(encoder_inputs, dtype=torch.long)
# enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
enc_emb = torch.nn.Embedding(num_embeddings= num_encoder_tokens -1, embedding_dim = latent_dim)
# encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_lstm = torch.nn.LSTM(input_size = num_encoder_tokens, hidden_size = latent_dim)
# encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
# encoder_states = [state_h, state_c]
# decoder_inputs = Input(shape=(None,))
# dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = torch.nn.Embedding(num_embeddings = num_decoder_tokens, embedding_dim = latent_dim)
# dec_emb = dec_emb_layer(decoder_inputs)
# dec_emb = dec_emb_layer(num_decoder_tokens, latent_dim)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
# decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_lstm = torch.nn.LSTM(input_size = num_encoder_tokens, hidden_size = latent_dim)
# decoder_outputs, _, _ = decoder_lstm(dec_emb,
#                                      initial_state=encoder_states)
# decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_dense = torch.nn.Linear(in_features = num_encoder_tokens, out_features = num_decoder_tokens)
activation = torch.nn.Softmax()
# decoder_outputs = decoder_dense(decoder_outputs)


# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
# model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
# model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
# model.summary()

train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 128
epochs = 100



In [20]:
# model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
#                     steps_per_epoch = train_samples//batch_size,
#                     epochs=epochs,
#                     validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
#                     validation_steps = val_samples//batch_size)



In [21]:
# Encode the input sequence to get the "thought vectors"
encoder_model = torch.nn.Sequential(enc_emb, encoder_lstm)
decoder_model = torch.nn.Sequential(dec_emb, decoder_lstm, decoder_dense, activation)
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    
    states_value = encoder_model(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [22]:
# k+=1
# (input_seq, actual_output), _ = next(train_gen)

# print(input_seq)
# len(input_seq)
# decoded_sentence = decode_sequence(torch.tensor(input_seq, dtype=torch.long))
# print('Input English sentence:', X_train[k:k+1].values[0])
# print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
# print('Predicted Hindi Translation:', decoded_sentence[:-4])

In [23]:
X_train

7001                                     or power to punish
100086                                     two months later
39185     they were sent down to find if there was any o...
46216     of building a greenway movement in the south b...
107962    and healthy babies screeching in the delivery ...
                                ...                        
119859                       i realized that people like me
18518          who is a man who sparked the psychedelic era
37799     but nevertheless neuroscience is turning up mo...
49669                                    which has happened
64791     im a little bit of an amateur historian so ive...
Name: english_sentence, Length: 19819, dtype: object

In [24]:
y_train

7001                    START_ या दण्डित करने की शक्ति _END
100086                             START_ दो महीने बाद _END
39185     START_ उन्हें वहाँ संभावनाओं की तलाश में भेजा ...
46216     START_ वह साऊथ ब्रोंकस में हरित क्रांति लाने क...
107962    START_ चिल्लाकर रोने जैसे गर्वीले लम्हों पर मु...
                                ...                        
119859                START_ मुझे लगा कि मेरे जैसे लोग _END
18518     START_ के युग की शुरूआत करने वाले व्यक्ति मेरे...
37799     START_ लेकिन फिर भी तंत्रिका विज्ञान अधिक विपण...
49669                                 START_ जो हुआ है _END
64791                        START_ मैं इतिहास को देखा _END
Name: hindi_sentence, Length: 19819, dtype: object

In [25]:
for epoch in range(epochs):
    for batch in train_gen:
        input_seq, actual_output = batch
        input_seq = torch.tensor(np.array(input_seq).astype(np.int64))
        print(input_seq.squeeze(1).shape)
        decoded_sentence = decode_sequence(input_seq.squeeze(1))
        print('Input English sentence:', X_train[k:k+1].values[0])
        print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
        print('Predicted Hindi Translation:', decoded_sentence[:-4])

torch.Size([2, 20])


IndexError: index out of range in self